In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

In [2]:
device = {
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.backends.mps.is_available()
    else 'cpu'  
}   
print(device)

{'cpu'}


In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Should show your GPU name


In [3]:
import torch
print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.9.0+cpu
CUDA version: None
CUDA available: False


In [3]:
# HyperParameters
batch_size = 8
block_size = 32
max_iters = 1000
learning_rate = 1e-4
eval_iters = 100
n_embd = 128
n_head = 8
n_layer = 1
dropout = 0.2


In [4]:
chars = ""
with open('vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)    

FileNotFoundError: [Errno 2] No such file or directory: 'vocab.txt'

In [ ]:
chars = ""
with open("vocab.txt" , 'r', encoding='utf-8') as f:
    f.read()
    chars = sorted(list(set(text)))
    
vocab_size = len(chars)

FileNotFoundError: [Errno 2] No such file or directory: 'vocab.txt'

In [ ]:
string_to_int = {ch:i for ch, i  in enumerate(chars)}
int_to_string = {ch:i for ch, i in enumerate(chars)}
encode = lambda s : ''.join(string_to_int[c] for c in s)
decode = lambda l : ''.join(int_to_string[c] for c in l)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

In [ ]:
def get_random_chunk(split):
    filename = "../wizards_of_oz.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)
            
            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size-1)
            
            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long )
            
        return data
    

In [ ]:
def get_batch(split) :
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[ i : i + block_size] for i in ix ])
    y = torch.stack([data[ i+1 : i + block_size] for i in ix ])
    return x, y

In [ ]:
class Head(nn.Module) : 
    """ one head of step attention """
    
    def __init__(self, head_size) :
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear()